In [1]:
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import os
import json
import time
import numpy as np
import fiona
from geopandas import GeoDataFrame



In [2]:
bay_county_names = ["Alameda", "ContraCosta", "Sonoma", "Solano", "SanMateo", "SantaClara", "SanFrancisco", "Marin","Napa"] 
analysis_years = ["2009", "2013", "2014", "2015", "2016", "2017"]
geo_data_path = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/CTs_geo_data_1/"


#### Concatenate all CTs in 9 counties for each year and writes the resulting data to csv

I am trying to locate pairs of census tracts that are geographically proximate and demographically similar except for their CRA eligibility threshold as the first step towards establishing a regression discontinuity design study. 

In [3]:
#concatenates all the census tracts in a given year from all the counties in the Bay Area. This is because when selectiing
#neighbors I do not want to eliminate census tracts that border each other but are separated by administrative 
#boundaries (ie: cities, counties), so I consider the congruous set of tracts for this analysis

for year in analysis_years:
    county_folders = [f for f in os.listdir("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/CTs_geo_data_1/")
                     if f[-3:] != ".DS_Store"
                     and f.split('_')[1] == (str(year))]    
    countygfs = []
    print(county_folders)
    
    for folder in county_folders:
        county = folder.split("_")[0]
        gdf = gp.read_file(geo_data_path+folder+"/"+county+"_geoparsed_"+year+".shp")
        countygfs.append(gdf)
    print(len(countygfs))
    try:
        to_write = pd.concat(countygfs, axis=0) #concatenates all the dataframes from that year together
        display(to_write.columns)
        combined_shapefile_path = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/combined_shapefiles/"
        to_write.to_file(driver='ESRI Shapefile', filename=combined_shapefile_path+"/all_bay_concat_"+year+"/all_bay_concat_"+year+".shp")
        print('{0} succeeded'.format(year))
    except Exception as e:
        print('{0} failed'.format(year))
    


['Marin_2009', 'SanMateo_2009', 'Napa_2009', 'Solano_2009', 'Sonoma_2009', 'SanFrancisco_2009', 'ContraCosta_2009', 'SantaClara_2009', 'Alameda_2009']
9


Index(['STATEFP00', 'COUNTYFP00', 'TRACTCE00', 'CTIDFP00', 'NAME00',
       'NAMELSAD00', 'MTFCC00', 'FUNCSTAT00', 'Tract', 'Year', 'County',
       'type', 'CRA Eligib', '% HI borro', '# HI borro', '% HI bor_1',
       '# HI bor_1', '% LI borro', '# LI borro', '%LI borrow', '# LI bor_1',
       'Geoid', 'geometry'],
      dtype='object')

2009 succeeded
['Solano_2013', 'Napa_2013', 'SanMateo_2013', 'Marin_2013', 'Alameda_2013', 'SantaClara_2013', 'ContraCosta_2013', 'SanFrancisco_2013', 'Sonoma_2013']
9


Index(['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAME10',
       'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10',
       'INTPTLAT10', 'INTPTLON10', 'NAME', 'Tract', 'Year', 'County', 'type',
       'CRA Eligib', '% HI borro', '# HI borro', '% HI bor_1', '# HI bor_1',
       '% LI borro', '# LI borro', '%LI borrow', '# LI bor_1', 'Geoid',
       'geometry'],
      dtype='object')

2013 succeeded
['Solano_2014', 'Napa_2014', 'SanMateo_2014', 'Marin_2014', 'Alameda_2014', 'SantaClara_2014', 'ContraCosta_2014', 'SanFrancisco_2014', 'Sonoma_2014']
9


...

2015 succeeded
['Alameda_2016', 'ContraCosta_2016', 'SantaClara_2016', 'Sonoma_2016', 'SanFrancisco_2016', 'Napa_2016', 'Solano_2016', 'SanMateo_2016', 'Marin_2016']
9


Index(['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAME10',
       'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10',
       'INTPTLAT10', 'INTPTLON10', 'NAME', 'Tract', 'Year', 'County', 'type',
       'CRA Eligib', '% HI borro', '# HI borro', '% HI bor_1', '# HI bor_1',
       '% LI borro', '# LI borro', '%LI borrow', '# LI bor_1', 'Geoid',
       'geometry'],
      dtype='object')

2016 succeeded
['Alameda_2017', 'SantaClara_2017', 'ContraCosta_2017', 'SanFrancisco_2017', 'Sonoma_2017', 'Solano_2017', 'Napa_2017', 'SanMateo_2017', 'Marin_2017']
9


Index(['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAME10',
       'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10',
       'INTPTLAT10', 'INTPTLON10', 'NAME', 'Tract', 'Year', 'County', 'type',
       'CRA Eligib', '% HI borro', '# HI borro', '% HI bor_1', '# HI bor_1',
       '% LI borro', '# LI borro', '%LI borrow', '# LI bor_1', 'Geoid',
       'geometry'],
      dtype='object')

2017 succeeded


#### Building Geographic Neighbors Dictionary

Takes in a geodataframe which has the HMDA information geolocated to polygons, concatenated for all of the 9 counties. Builds a dictionary where the keys are each unique tract number and the values are an array of geographic neighbors 
of that tract, and the eligibility threshold does not match the tract in question. The first function "find_neighbors_all" finds all of the CTs that are geographic neighbors and are oppositely CRA coded, the second function "find_neighbors_midmod" finds neighbors only for those tracts that are middle or moderate income, by eliminating high (>120% AMI) and low (<50% AMI) tracts.

In [4]:
def find_neighbors_all(geodataframe, year):
    neighbors_dict = {}
    for df_row in geodataframe.iterrows(): #index, then row
        polygon = df_row[1]["geometry"]
        

        eligibility = df_row[1]["CRA Eligib"]
        if (year == '2009'):
            tract = (df_row[1]["CTIDFP00"])
            neighbors_dict[tract] = list(geodataframe[geodataframe.apply(lambda row: row['geometry'].touches(polygon) and 
            row['CRA Eligib'] != eligibility, axis=1)]["CTIDFP00"].values)#list so that it is easier to write to json later
        else:
            tract = (df_row[1]["GEOID10"])
            neighbors_dict[tract] = list(geodataframe[geodataframe.apply(lambda row: row['geometry'].touches(polygon) and 
            row['CRA Eligib'] != eligibility, axis=1)]["GEOID10"].values)#list so that it is easier to write to json later
  
    return neighbors_dict

    
    


In [5]:
def find_neighbors_midmod(geodataframe, year):
    neighbors_dict = {}
    geodf_mid_mod = geodataframe[(geodataframe['type'] == "mod") | (geodataframe["type"] == "mid")]
    for df_row in geodf_mid_mod.iterrows(): #index, then row
        polygon = df_row[1]["geometry"]
        

        eligibility = df_row[1]["CRA Eligib"]
        if (year == '2009'):
            tract = (df_row[1]["CTIDFP00"])
            neighbors_dict[tract] = list(geodf_mid_mod[geodf_mid_mod.apply(lambda row: row['geometry'].touches(polygon) and 
                    row['CRA Eligib'] != eligibility, axis=1)]["CTIDFP00"].values)#list so that it is easier to write to json later
        else:
            tract = (df_row[1]["GEOID10"])
            neighbors_dict[tract] = list(geodf_mid_mod[geodf_mid_mod.apply(lambda row: row['geometry'].touches(polygon) and 
                    row['CRA Eligib'] != eligibility, axis=1)]["GEOID10"].values)#list so that it is easier to write to json later
    return neighbors_dict

In [6]:
for year in ["2016"]:
    geodf = gp.read_file('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/combined_shapefiles/all_bay_concat_'+year+'/all_bay_concat_'+year+'.shp')
    display(len(geodf))
    
    

1559

#### This takes a very long time to run, but since both the all tracts and mid mod dictionaries are saves as npy files it shouldn't have to be run repeatedly

In [7]:
start = time.time()
neighbors_years = {}
neighbors_years_midmod = {}


for year in analysis_years:
    #reads in a shapefile of all the CTs in the 9 counties of a given year
    geodf = gp.read_file('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/combined_shapefiles/all_bay_concat_'+year+'/all_bay_concat_'+year+'.shp')
    #applies the find neighbors function from above
#     print("finding mid mod neighbors for "+ str(year))

#     neighbors_years_midmod[year] = find_neighbors_midmod(geodf, year) #for only mid mod tracts
    print("finding all neighbors for "+ str(year))

    neighbors_years[year] = find_neighbors_all(geodf, year) #adds the dictionary of neighbors for a given year to the master
    #dictionary of neighbors for all of the years
    

#writes the dictionary to json, if needed
#     with open('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors_all_'+str(year)+'.json', 'w') as fp:
#         json.dump(neighbors, fp)
end =time.time()
elapsed = end-start

elapsed

finding all neighbors for 2009
finding all neighbors for 2013
finding all neighbors for 2014
finding all neighbors for 2015
finding all neighbors for 2016
finding all neighbors for 2017


473.92005705833435

#### Saves the neighbors dictionary as a numpy object, to be read in later as a dictionary


In [18]:
np.save('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/neighbors_all_allyears_new.npy', neighbors_years)



In [31]:
# np.save('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/neighbors_midmod_allyears_new.npy', neighbors_years_midmod)

#### Load saved npy files with neighbors

In [32]:
neighbors_years = np.load('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/neighbors_all_allyears_new.npy').item()
neighbors_years_midmod = np.load('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/neighbors_midmod_allyears_new.npy').item()



# print("num mid mod tracts 2009 ", len(neighbors_years_midmod['2009'].values())) #910 tracts in the midmod dict
print("num all tracts 2009", len(neighbors_years["2009"].values())) # 1397 tracts in the all dict

# print("num mid mod tracts 2013", len(neighbors_years_midmod['2013'].values())) #910 tracts in the midmod dict
print("num all tracts 2013", len(neighbors_years['2013'].values())) # 1397 tracts in the all dict



num all tracts 2009 1375
num all tracts 2013 1533


#### Creates a dataframe of all the tracts with neighbors by year (2009 to start), indicating which are CRA eligible. Exports to csv called "boolean_tracts_year"


In [33]:
ntracts = []
years = []
master = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/reference/tracts_type_master_1.csv')
boolean_tracts = pd.DataFrame()
for year in ['2013']: #- this is for the recovery models, 2013-2017. they use the 2010 tract categorizations 
    for tract in neighbors_years[year]:
        if (len(neighbors_years[year][tract]) > 0): #if the tract has neighbors
            ntracts.append(tract) #add it to the list "ntracts"
            years.append(year) #and add the year to the years list
    cra_ref = master[master["Year"] == int(year)] #reference dataframe for just the year in question
            

boolean_tracts["Tract"] = [int(tract) for tract in ntracts]#add the column 'Tract' to boolean_tracts of all of th tracts with neighbors
boolean_tracts["Year"] = years #add the column 'Year' to boolean_tracts of all of the years
# merge the boolean_tracts_all dataframe (all the tracts with neighbors), with their respective CRA coding
# based on the reference dataframe
boolean_tracts_withcra = pd.merge(boolean_tracts, cra_ref,  how='inner', left_on="Tract", right_on ='Geoid')
boolean_tracts_withcra = boolean_tracts_withcra[["Geoid", "Year_x", "Tract_y", "type", "County", "CRA Eligible"]]


boolean_tracts_withcra.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/boolean_tracts_all_2010.csv", index=False)
print("num all tracts that have at least one oppositely coded neighbor", len(boolean_tracts_withcra))


num all tracts that have at least one oppositely coded neighbor 968


In [12]:
# ntracts = []
# years = []
# master = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/reference/tracts_type_master_1.csv')
# boolean_tracts_midmod = pd.DataFrame()
# for year in ["2013"]:
#     for tract in neighbors_years_midmod[year]:
#         if (len(neighbors_years_midmod[year][tract]) > 0): #if the tract has neighbors
#             ntracts.append(tract) #add it to the list "ntracts"
#             years.append(year) #and add the year to the years list
#     cra_ref_midmod = master[master["Year"] == int(year)] #reference dataframe for just the year in question
            

# boolean_tracts_midmod["Tract"] = [int(tract) for tract in ntracts] #add the column 'Tract' to boolean_tracts of all of th tracts with neighbors
# boolean_tracts_midmod["Year"] = years #add the column 'Year' to boolean_tracts of all of the years


# #merge the boolean_tracts_all dataframe (all the tracts with neighbors), with their respective CRA coding
# #based on the reference dataframe
# boolean_tracts_midmod_withcra = pd.merge(boolean_tracts_midmod, cra_ref_midmod,  how='inner', left_on="Tract", right_on ='Geoid')
# # boolean_tracts_midmod_withcra.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/boolean_tracts_midmod_2009.csv", index=False)
# boolean_tracts_midmod_withcra.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/boolean_tracts_midmod_2010.csv", index=False)


# print("num mid or mod tracts that have at least one oppositely coded neighbor", len(boolean_tracts_midmod))


KeyError: '2013'

In [ ]:
# ntracts = []
# years = []
# boolean_tracts_midmod = pd.DataFrame()
# for year in ["2009"]:
#     for tract in neighbors_years_midmod[year]:
#         if (len(neighbors_years_midmod[year][tract]) > 0): #if the tract has neighbors
#             ntracts.append(tract) #add it to the list "ntracts"
#             years.append(year) #and add the year to the years list
#     cra_ref_midmod = master[master["Year"] == int(year)] #reference dataframe for just the year in question
            

# boolean_tracts_midmod["Tract"] = [int(tract) for tract in ntracts] #add the column 'Tract' to boolean_tracts of all of th tracts with neighbors
# boolean_tracts_midmod["Year"] = years #add the column 'Year' to boolean_tracts of all of the years


# #merge the boolean_tracts_all dataframe (all the tracts with neighbors), with their respective CRA coding
# #based on the reference dataframe
# boolean_tracts_midmod_withcra = pd.merge(boolean_tracts_midmod, cra_ref_midmod,  how='inner', left_on="Tract", right_on ='Geoid')
# # boolean_tracts_midmod_withcra.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/boolean_tracts_midmod_2009.csv", index=False)
# boolean_tracts_midmod_withcra.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/boolean_tracts_midmod_2009.csv", index=False)


# print("num mid or mod tracts that have at least one oppositely coded neighbor", len(boolean_tracts_midmod))


In [35]:
ntracts = []
years = []
master = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/reference/tracts_type_master_1.csv')
boolean_tracts = pd.DataFrame()
for year in ['2009']: #this is for all the models to avoid endogeneity 
    for tract in neighbors_years[year]:
        if (len(neighbors_years[year][tract]) > 0): #if the tract has neighbors
            ntracts.append(tract) #add it to the list "ntracts"
            years.append(year) #and add the year to the years list
    cra_ref = master[master["Year"] == int(year)] #reference dataframe for just the year in question
            

boolean_tracts["Tract"] = [int(tract) for tract in ntracts]#add the column 'Tract' to boolean_tracts of all of th tracts with neighbors
boolean_tracts["Year"] = years #add the column 'Year' to boolean_tracts of all of the years
# merge the boolean_tracts_all dataframe (all the tracts with neighbors), with their respective CRA coding
# based on the reference dataframe
boolean_tracts_withcra = pd.merge(boolean_tracts, cra_ref,  how='inner', left_on="Tract", right_on ='Geoid')
boolean_tracts_withcra = boolean_tracts_withcra[["Geoid", "Year_x", "Tract_y", "type", "County", "CRA Eligible"]]


boolean_tracts_withcra.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/boolean_tracts_all_2009.csv", index=False)
print("num all tracts that have at least one oppositely coded neighbor", len(boolean_tracts_withcra))


num all tracts that have at least one oppositely coded neighbor 792
